<a href="https://colab.research.google.com/github/woolozzx21/leesuan/blob/main/OpenCV_Python_%EC%98%81%EC%83%81_%EC%9D%B4%EC%A7%84%ED%99%94%2C_%ED%9E%88%EC%8A%A4%ED%86%A0%EA%B7%B8%EB%9E%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 실습용 데이터 다운로드
!git clone https://github.com/JinFree/OpenCV_in_Ubuntu

# 기본 라이브러리 Import
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

In [ ]:
# 컬러 이미지 열기
image_color = cv2.imread("/content/OpenCV_in_Ubuntu/Data/Lane_Detection_Images/solidWhiteCurve.jpg", cv2.IMREAD_COLOR)
cv2_imshow(image_color)
# 회색조 이미지 열기
image_gray = cv2.imread("/content/OpenCV_in_Ubuntu/Data/Lane_Detection_Images/solidWhiteCurve.jpg", cv2.IMREAD_GRAYSCALE)
cv2_imshow(image_gray)

###영상 이진화


```
thresh = 128
maxval = 255
flag = cv2.THRESH_BINARY
ret, dst_image = cv2.threshold(src_image, thresh, maxval, flag)
단, src_image는 1채널 회색조 이미지
```

[threshold flags](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)

In [ ]:
image = np.zeros((512, 512), np.uint8)
for i in range(0, 512):
    for j in range(0, 256):
        image[i, j] = j
    for j in range(256, 512):
        image[i, j] = 255-j
cv2_imshow(image)

In [ ]:
thresh = 128
maxval = 255
flag = cv2.THRESH_BINARY
ret, dst_image = cv2.threshold(image, thresh, maxval, flag)
cv2_imshow(dst_image)

In [ ]:
flag = cv2.THRESH_BINARY_INV
ret, dst_image = cv2.threshold(image, thresh, maxval, flag)
cv2_imshow(dst_image)

In [ ]:
flag = cv2.THRESH_TRUNC
ret, dst_image = cv2.threshold(image, thresh, maxval, flag)
cv2_imshow(dst_image)

In [ ]:
flag = cv2.THRESH_TOZERO
ret, dst_image = cv2.threshold(image, thresh, maxval, flag)
cv2_imshow(dst_image)

In [ ]:
flag = cv2.THRESH_TOZERO_INV
ret, dst_image = cv2.threshold(image, thresh, maxval, flag)
cv2_imshow(dst_image)

###히스토그램
히스토그램 계산 후 그리는 함수


```
def computeHist(image, mask=None):
    bins = np.arange(256).reshape(256,1)
    if len(image.shape)==2:
        h = np.zeros((300,256,1))
        hist_item = cv2.calcHist([image],[0],None,[256],[0,255]) 
        cv2.normalize(hist_item,hist_item,0,255,cv2.NORM_MINMAX) 
        hist=np.int32(np.around(hist_item)) 
        pts = np.column_stack((bins,hist)) 
        cv2.polylines(h,[pts],False, 255)
    else:
        h = np.zeros((300,256,3))
        color = [ (255,0,0),(0,255,0),(0,0,255) ] 
        for ch, col in enumerate(color): 
            hist_item = cv2.calcHist([image],[ch],None,[256],[0,255]) 
            cv2.normalize(hist_item,hist_item,0,255,cv2.NORM_MINMAX) 
            hist=np.int32(np.around(hist_item)) 
            pts = np.column_stack((bins,hist)) 
            cv2.polylines(h,[pts],False,col)
    return np.flipud(h)
```



In [ ]:
def computeHist(image, mask=None):
    bins = np.arange(256).reshape(256,1)
    if len(image.shape)==2:
        h = np.zeros((300,256,1))
        hist_item = cv2.calcHist([image],[0],None,[256],[0,255]) 
        cv2.normalize(hist_item,hist_item,0,255,cv2.NORM_MINMAX) 
        hist=np.int32(np.around(hist_item)) 
        pts = np.column_stack((bins,hist)) 
        cv2.polylines(h,[pts],False, 255)
    else:
        h = np.zeros((300,256,3))
        color = [ (255,0,0),(0,255,0),(0,0,255) ] 
        for ch, col in enumerate(color): 
            hist_item = cv2.calcHist([image],[ch],None,[256],[0,255]) 
            cv2.normalize(hist_item,hist_item,0,255,cv2.NORM_MINMAX) 
            hist=np.int32(np.around(hist_item)) 
            pts = np.column_stack((bins,hist)) 
            cv2.polylines(h,[pts],False,col)
    return np.flipud(h)

In [ ]:
color_histogram = computeHist(image_color)
cv2_imshow(image_color)
cv2_imshow(color_histogram)

In [ ]:
gray_histogram = computeHist(image_gray)
cv2_imshow(image_gray)
cv2_imshow(gray_histogram)

In [ ]:
image_hsv = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)
hsv_histogram = computeHist(image_hsv)
cv2_imshow(image_hsv)
cv2_imshow(hsv_histogram)

In [ ]:
image_hls = cv2.cvtColor(image_color, cv2.COLOR_BGR2HLS)
hls_histogram = computeHist(image_hls)
cv2_imshow(image_hls)
cv2_imshow(hls_histogram)

###히스토그램 균일화


```
히스토그램 균일화
dst_image = cv2.equalizeHist(src_image)

적응형 히스토그램 균일화
clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
dst_image = clahe.apply(src_image)

단, 히스토그램 균일화는 회색조 이미지만 가능
```




In [ ]:
gray_equalized = cv2.equalizeHist(image_gray)
cv2_imshow(gray_equalized)

In [ ]:
image_hsv = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(image_hsv)

h_eq = cv2.equalizeHist(h)
s_eq = cv2.equalizeHist(s)
v_eq = cv2.equalizeHist(v)

image_hseqveq = cv2.merge((h, s_eq, v_eq))
image_hsveq = cv2.merge((h, s, v_eq))
image_hseqv = cv2.merge((h, s_eq, v))

bgr_hseqveq = cv2.cvtColor(image_hseqveq, cv2.COLOR_HSV2BGR)
bgr_hsveq = cv2.cvtColor(image_hsveq, cv2.COLOR_HSV2BGR)
bgr_hseqv = cv2.cvtColor(image_hseqv, cv2.COLOR_HSV2BGR)

cv2_imshow(bgr_hseqveq)
cv2_imshow(bgr_hsveq)
cv2_imshow(bgr_hseqv)

In [ ]:
image_hsv = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(image_hsv)

clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
h_eq = clahe.apply(h)
s_eq = clahe.apply(s)
v_eq = clahe.apply(v)

image_hseqveq = cv2.merge((h, s_eq, v_eq))
image_hsveq = cv2.merge((h, s, v_eq))
image_hseqv = cv2.merge((h, s_eq, v))

bgr_hseqveq = cv2.cvtColor(image_hseqveq, cv2.COLOR_HSV2BGR)
bgr_hsveq = cv2.cvtColor(image_hsveq, cv2.COLOR_HSV2BGR)
bgr_hseqv = cv2.cvtColor(image_hseqv, cv2.COLOR_HSV2BGR)

cv2_imshow(bgr_hseqveq)
cv2_imshow(bgr_hsveq)
cv2_imshow(bgr_hseqv)

### 실습

/content/OpenCV_in_Ubuntu/Data/Lane_Detection_Images/solidWhiteCurve.jpg

위 경로의 이미지를 연 후, 아래 작업을 수행하시오.
1. 이미지를 hls 색공간으로 바꾼 후, s채널에 대해 적응형 히스토그램 평활화를 수행하시오.
2. 평활화된 s 채널에 대해 임계값 200, 최대값 255로 cv2.THRESH_BINARY를 수행하시오.
3. l채널에 대해 임계값 110, 최대값 128로 cv2.THRESH_BINARY를 수행하시오.
3. 이후 각 채널을 합친 후 bgr로 변환하여 확인하시오.

In [ ]:
### 여기에 실습을 진행하세요.